In [2]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
netflix_df = pd.read_csv('../data/netflix-pre-processed.csv')
netflix_df.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,day_added,year_added,month_added
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Cast,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",25,2021,9
1,1,s2,TV Show,Blood & Water,No Director,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",24,2021,9
2,2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Country,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,24,2021,9
3,3,s4,TV Show,Jailbirds New Orleans,No Director,No Cast,No Country,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",24,2021,9
4,4,s5,TV Show,Kota Factory,No Director,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,24,2021,9


In [4]:
netflix_df.shape

(8790, 16)

In [5]:
netflix_df.columns

Index(['Unnamed: 0', 'show_id', 'type', 'title', 'director', 'cast', 'country',
       'date_added', 'release_year', 'rating', 'duration', 'listed_in',
       'description', 'day_added', 'year_added', 'month_added'],
      dtype='object')

In [6]:
# Filter the DataFrame to only include movies
movies_df = netflix_df[netflix_df['type'] == 'Movie']

In [7]:
# Replace "[" and "]" characters in the "listed_in" column with empty strings
movies_df['listed_in'] = movies_df['listed_in'].str.replace('[\[\]]', '')
movies_df.head()

<>:2: DeprecationWarning: invalid escape sequence \[
<>:2: DeprecationWarning: invalid escape sequence \[
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_11268\3938948011.py:2: DeprecationWarning: invalid escape sequence \[
  movies_df['listed_in'] = movies_df['listed_in'].str.replace('[\[\]]', '')
C:\Users\SQ-PC\AppData\Local\Temp\ipykernel_11268\3938948011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['listed_in'] = movies_df['listed_in'].str.replace('[\[\]]', '')


,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,day_added,year_added,month_added
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Cast,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",25,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9
7,7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",24,2021,9
9,9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,24,2021,9
12,12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021-09-23,2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...,23,2021,9


In [8]:
# Split the "listed_in" column into separate rows for each genre
genres_df = movies_df.assign(listed_in=movies_df['listed_in'].str.split(',')).explode('listed_in')
genres_df['listed_in'] = genres_df['listed_in'].str.strip()
genres_df.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,day_added,year_added,month_added
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Cast,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",25,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9
7,7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,Dramas,"On a photo shoot in Ghana, an American model s...",24,2021,9
7,7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,Independent Movies,"On a photo shoot in Ghana, an American model s...",24,2021,9
7,7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,International Movies,"On a photo shoot in Ghana, an American model s...",24,2021,9


In [9]:
# Split the "director" and "cast" columns into separate rows for each director and cast member
directors_df = genres_df.assign(director=genres_df['director'].str.split(',')).explode('director')
directors_df.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,day_added,year_added,month_added
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Cast,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",25,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,Robert Cullen,"Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,José Luis Ucha,"Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9
7,7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,Dramas,"On a photo shoot in Ghana, an American model s...",24,2021,9
7,7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",2021-09-24,1993,TV-MA,125 min,Independent Movies,"On a photo shoot in Ghana, an American model s...",24,2021,9


In [10]:
cast_df = directors_df.assign(cast=directors_df['cast'].str.split(',')).explode('cast')
cast_df.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,day_added,year_added,month_added
0,0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,No Cast,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",25,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,Robert Cullen,Vanessa Hudgens,No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,Robert Cullen,Kimiko Glenn,No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,Robert Cullen,James Marsden,No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9
6,6,s7,Movie,My Little Pony: A New Generation,Robert Cullen,Sofia Carson,No Country,2021-09-24,2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,24,2021,9


In [11]:
# Convert the DataFrame into a list of transactions
transactions = cast_df.groupby(['director', 'cast', 'listed_in'])['title'].apply(list).reset_index(name='movies')
transactions.head()

,director,cast,listed_in,movies
0,Aaron Moorhead,Ally Ioannides,Thrillers,[Synchronic]
1,Aaron Moorhead,Bill Oberst Jr.,Thrillers,[Synchronic]
2,Aaron Moorhead,Callie Hernandez,Independent Movies,[The Endless]
3,Aaron Moorhead,Callie Hernandez,Sci-Fi & Fantasy,[The Endless]
4,Aaron Moorhead,Callie Hernandez,Thrillers,[The Endless]


In [12]:
# Convert the list of transactions into a one-hot encoded format
te = TransactionEncoder()
te_ary = te.fit(transactions['movies']).transform(transactions['movies'])
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

,#Alive,#AnneFrank - Parallel Stories,#FriendButMarried,#FriendButMarried 2,#Roxy,#Rucker50,#Selfie,#Selfie 69,#cats_the_mewvie,#realityhigh,...,İstanbul Kırmızısı,اشتباك,الف مبروك,يوم الدين,​Goli Soda 2,​Maj Rati ​​Keteki,​Mayurakshi,​​Kuch Bheege Alfaaz,반드시 잡는다,최강전사 미니특공대 : 영웅의 탄생
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
# Find frequent itemsets with a minimum support of 0.001
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.001232,(Ajeeb Daastaans)
1,0.001046,(Avengers: Infinity War)
2,0.001008,(Circle)
3,0.001400,(Deep)
4,0.001867,(Don Quixote: The Ingenious Gentleman of La Ma...


In [14]:
# Generate association rules with a minimum confidence of 0.5
association_rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5)
association_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


In [15]:
# Print the association rules
print(association_rules)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
